In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [98]:
# Load Datasets
assessment = pd.read_csv('Datasets/assessments.csv')
courses = pd.read_csv('Datasets/courses.csv')
student_assess = pd.read_csv('Datasets/studentAssessment.csv')
student_info = pd.read_csv('Datasets/studentInfo.csv')
student_regis = pd.read_csv('Datasets/studentRegistration.csv')
student_vle = pd.read_csv('Datasets/studentVle.csv')
vle = pd.read_csv('Datasets/vle.csv')

In [99]:
# Create a new feature "Module" to combine the feature 'code_module' with 'code_presentation'.
assessment['module_presentation'] = list(map(lambda a,b: str(a) + '_' + str(b), assessment.code_module, assessment.code_presentation))
courses['module_presentation'] = list(map(lambda a,b: str(a) + '_' + str(b), courses.code_module, courses.code_presentation))
student_info['module_presentation'] = list(map(lambda a,b: str(a) + '_' + str(b), student_info.code_module, student_info.code_presentation))
student_regis['module_presentation'] = list(map(lambda a,b: str(a) + '_' + str(b), student_regis.code_module, student_regis.code_presentation))
vle['module_presentation'] = list(map(lambda a,b: str(a)+ '_' + str(b), vle.code_module, vle.code_presentation))


In [100]:
#Menangani missing values dengan menginput berupa '0'
#courses.fillna(0, inplace=True)
#ssessment.fillna(0, inplace=True)
#vle.fillna(0, inplace=True)
#student_info.fillna(0, inplace=True)
#student_regis.fillna(0, inplace=True)
#student_assess.fillna(0, inplace=True)
#student_vle.fillna(0, inplace=True)

Preprocessing dan Integrasi/Merge

In [101]:
# The features "week_from" and "week_to" are dropped because the instances in these features are almost all null.
vle.drop(columns=['week_from', 'week_to'], inplace=True)

In [102]:
# Data consistency check with the final_result feature on the studentInfo entity

var_temp = student_regis.loc[student_regis.date_unregistration.notna(),['id_student', 'module_presentation', 'date_unregistration']]

var_temp = pd.merge(student_info, var_temp, on=['id_student', 'module_presentation'])

# Displays unregistered students without "Withdrawn" in the final_result feature.
err_final_result = var_temp.loc[var_temp.final_result != 'Withdrawn']
incorrect_final_results = err_final_result.index
err_final_result

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation,date_unregistration
719,BBB,2013J,362907,F,South West Region,Lower Than A Level,20-30%,35-55,2,60,N,Fail,BBB_2013J,0.0
724,BBB,2013J,365288,F,South Region,A Level or Equivalent,70-80%,0-35,0,60,N,Fail,BBB_2013J,0.0
869,BBB,2013J,554243,F,South West Region,Lower Than A Level,60-70%,0-35,0,60,N,Fail,BBB_2013J,166.0
4961,DDD,2013J,315082,M,West Midlands Region,A Level or Equivalent,50-60%,0-35,0,60,N,Fail,DDD_2013J,0.0
5010,DDD,2013J,403052,F,Yorkshire Region,HE Qualification,20-30%,0-35,0,150,N,Fail,DDD_2013J,0.0
5293,DDD,2013J,582954,M,London Region,Lower Than A Level,10-20,0-35,0,120,N,Fail,DDD_2013J,-7.0
7853,FFF,2013J,234004,M,Yorkshire Region,A Level or Equivalent,50-60%,0-35,0,60,N,Fail,FFF_2013J,0.0
8001,FFF,2013J,523777,M,North Region,A Level or Equivalent,0-10%,35-55,1,60,N,Fail,FFF_2013J,0.0
8340,FFF,2013J,601640,M,London Region,HE Qualification,20-30%,0-35,0,90,N,Fail,FFF_2013J,-4.0


In [103]:
# Do the 'Withdrawn' input process on students that do not have the 'Withdrawn' label
for row in err_final_result[['id_student','code_module','code_presentation']].values:
    student_info.loc[(student_info.id_student==row[0])&(student_info.code_module==row[1])&(student_info.code_presentation==row[2]), 'final_result'] = 'Withdrawn'

In [104]:
# Checking the weight on the assessment and presentation module.
grouped_assessments = assessment.groupby(['code_module', 'code_presentation']).agg(total_weight = ('weight', sum))
grouped_assessments.reset_index(inplace=True)
grouped_assessments

,code_module,code_presentation,total_weight
0,AAA,2013J,200.0
1,AAA,2014J,200.0
2,BBB,2013B,200.0
3,BBB,2013J,200.0
4,BBB,2014B,200.0
5,BBB,2014J,200.0
6,CCC,2014B,300.0
7,CCC,2014J,300.0
8,DDD,2013B,200.0
9,DDD,2013J,200.0


In [105]:
filtered_assessments = assessment[(assessment['code_module'].isin(["CCC", "GGG"]))]

# Group by specified columns and aggregate the sum of 'weight' as 'type_weights'
grouped_assessments = filtered_assessments.groupby(['code_module', 'code_presentation', 'assessment_type']) \
    .agg(type_weights=('weight', 'sum'))

# Reset index to make the DataFrame more readable
grouped_assessments.reset_index(inplace=True)

grouped_assessments

,code_module,code_presentation,assessment_type,type_weights
0,CCC,2014B,CMA,25.0
1,CCC,2014B,Exam,200.0
2,CCC,2014B,TMA,75.0
3,CCC,2014J,CMA,25.0
4,CCC,2014J,Exam,200.0
5,CCC,2014J,TMA,75.0
6,GGG,2013J,CMA,0.0
7,GGG,2013J,Exam,100.0
8,GGG,2013J,TMA,0.0
9,GGG,2014B,CMA,0.0


Pada result code diatas terdapat weight yang error, di code_modul CCC dan GGG

In [106]:
#Weight Exam dibagi 2 untuk modul CCC
assessment.loc[(assessment['code_module'] == 'CCC') & (assessment['assessment_type'] == 'Exam'), 'weight'] /= 2

#Weight penilain TMA diupdate sebesar 100%
assessment.loc[(assessment['code_module'] == 'GGG') & (assessment['assessment_type'] == 'TMA'), 'weight'] = 100 / 3

In [107]:
# Merge the two tables on 'id_assessment' and calculate 'score*weight'
merged_table = pd.merge(student_assess, assessment, on='id_assessment', how='left')
merged_table['score*weight'] = (merged_table['score'] * merged_table['weight'])

# Group by specified columns and aggregate sum of 'score*weight' and 'weight'
grouped_marks = merged_table.groupby(['id_student', 'code_module', 'code_presentation'], as_index=False)[['score*weight', 'weight']].sum()

# Calculate adjusted_mark and mark
grouped_marks['adjusted_mark'] = grouped_marks['score*weight'] / grouped_marks['weight']
grouped_marks['mark'] = grouped_marks['score*weight'] / 200

# Rename columns
grouped_marks.rename(columns={'total_score_weight': 'total_score*weight',
                              'weight': 'attempted_weight'}, inplace=True)

# Round off values
grouped_marks = grouped_marks.round(1)

In [108]:
# Merge marks and info_student to obtain a larger table with relevant information
merged_table = pd.merge(grouped_marks, student_info, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Replace mark and adjusted_mark with NaN for rows where final_result is 'Withdrawn'
merged_table.loc[merged_table.final_result=='Withdrawn','mark']= np.nan
merged_table.loc[merged_table.final_result=='Withdrawn','adjusted_mark']= np.nan
#merged_table.loc[merged_table['final_result'] == 'Withdrawn', ['mark', 'adjusted_mark']] = np.nan

In [109]:
# Categories based on final_result
distinction = merged_table[merged_table.final_result == "Distinction"].index
passing = merged_table[merged_table.final_result == "Pass"].index
fail = merged_table[merged_table.final_result == "Fail"].index
withdraw = merged_table[merged_table.final_result == "Withdraw"].index

# Categories based on attempted_weight
attempt_weight200 = merged_table[merged_table.attempted_weight == 200].index
attempt_weight150_200 = merged_table[(merged_table.attempted_weight >= 150) & (merged_table.attempted_weight != 200)].index
attempt_weight0_150 = merged_table.index.difference(attempt_weight200).difference(attempt_weight150_200)

# Categories based on adjusted_mark
adj_mark80_100 = merged_table[merged_table.attempted_weight >= 80].index
adj_mark70_80 = merged_table[(merged_table.attempted_weight >= 70) & (merged_table.attempted_weight < 80)].index
adj_mark0_70 = merged_table[merged_table.attempted_weight < 70].index

# Categories based on mark
mark40_100 = merged_table[merged_table.attempted_weight >= 40].index
mark0_40 = merged_table[merged_table.attempted_weight < 40].index

print("Shape of merged_table:", merged_table.shape)

Shape of merged_table: (25843, 17)


In [111]:
# Get common indices for Distinction
distinction_indices = list(set(attempt_weight200).intersection(adj_mark80_100))
merged_table.loc[distinction_indices, "final_result"] = "Distinction"

# Get common indices for Pass
pass_indices = list(set(attempt_weight200).intersection(adj_mark70_80))
merged_table.loc[pass_indices, "final_result"] = "Pass"

# Get common indices for Fail
fail_indices = list(set(attempt_weight200).intersection(adj_mark0_70))
merged_table.loc[fail_indices, "final_result"] = "Fail"

# Get common indices for Attempted Weight between 150 and 200, and update final_result and adjusted_mark
common_indices = list(set(attempt_weight150_200).intersection(passing).intersection(mark40_100))
merged_table.loc[common_indices, "final_result"] = "Distinction"
merged_table.loc[common_indices, "adjusted_mark"] = merged_table.loc[common_indices, "mark"]

# Get common indices for Attempted Weight less than 150, and update adjusted_mark based on categories
attempt_weight0_150_indices = list(attempt_weight0_150)
distinction_indices_0_150 = list(set(attempt_weight0_150_indices).intersection(distinction))
pass_indices_0_150 = list(set(attempt_weight0_150_indices).intersection(passing))
fail_indices_0_150 = list(set(attempt_weight0_150_indices).intersection(fail))

# Sort the indices based on mark and update adjusted_mark accordingly
merged_table.loc[sorted(distinction_indices_0_150, key=lambda i: merged_table.loc[i, "mark"]), "adjusted_mark"] = np.arange(70.0, 100.0, 30/len(distinction_indices_0_150))
merged_table.loc[sorted(pass_indices_0_150, key=lambda i: merged_table.loc[i, "mark"]), "adjusted_mark"] = np.arange(40.0, 70.0, 30/len(pass_indices_0_150))
merged_table.loc[sorted(fail_indices_0_150, key=lambda i: merged_table.loc[i, "mark"]), "adjusted_mark"] = np.arange(0.0, 40.0, 40/len(fail_indices_0_150))

df = merged_table